This is taken from lesson 2 in https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/blob/master/Chapter02/

In [1]:
! wget "https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter02/Resources/test_data.npz"
! wget "https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter02/Resources/training_data.npz"
! wget "https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter02/Resources/test_labels.npy"
! wget "https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter02/Resources/training_labels.npy"

--2021-05-02 05:34:19--  https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter02/Resources/test_data.npz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter02/Resources/test_data.npz [following]
--2021-05-02 05:34:20--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter02/Resources/test_data.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38887 (38K) [application/octet-stream]
Saving to: ‘test_data.npz’

test_data.npz    

In [2]:
!pip install sklearn imbalanced-learn

In [3]:
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
import numpy as np
import scipy.sparse 
import collections

In [4]:
X_train = scipy.sparse.load_npz("training_data.npz")
y_train = np.load("training_labels.npy")
X_test = scipy.sparse.load_npz("test_data.npz")
y_test = np.load("test_labels.npy")

In [5]:
X_train, y_train, X_test, y_test

(<248x2097153 sparse matrix of type '<class 'numpy.float64'>'
 	with 13990 stored elements in COOrdinate format>,
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1], dtype=int32),
 <131x2097153 sparse matrix of 

In [6]:
dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
print(collections.Counter(dt_pred))
print(balanced_accuracy_score(y_test, dt_pred))

Counter({0: 120, 1: 11})
0.8290229885057472


## we can try several ways to improve the performace

### Weighting 

In [7]:
dt_weighted = tree.DecisionTreeClassifier(class_weight="balanced")
dt_weighted.fit(X_train, y_train)
dt_weighted_pred = dt_weighted.predict(X_test)
print(collections.Counter(dt_weighted_pred))
print(balanced_accuracy_score(y_test, dt_weighted_pred))

Counter({0: 114, 1: 17})
0.9913793103448276


### Upsampling the minorclass

In [8]:
from sklearn.utils import resample

In [9]:
X_train_np = X_train.toarray()
class_0_indices = [i for i, x in enumerate(y_train == 0) if x]
class_1_indices = [i for i, x in enumerate(y_train == 1) if x]
size_class_0 = sum(y_train == 0)


x_train_class_0 = X_train_np[class_0_indices, :]
y_train_class_0 = [0] * size_class_0
x_train_class_1 = X_train_np[class_1_indices, :]
y_train_class_1 = [1] * (len(y_train) - size_class_0)

In [10]:
# We upsample the elements of class 1 with replacements until the number ofsamples of class 1 and class 0 are equal
X_train_class_1_resampled = resample(x_train_class_1, replace=True, n_samples=size_class_0)
y_train_class_1_resampled = [1] * size_class_0

In [11]:
# combining
X_train_resampled = np.concatenate([x_train_class_0, X_train_class_1_resampled])
y_train_resampled = y_train_class_0 + y_train_class_1_resampled

In [15]:
# training a random forest
from scipy import sparse

X_train_resampled = sparse.csr_matrix(X_train_resampled)
dt_resampled = tree.DecisionTreeClassifier()
dt_resampled.fit(X_train_resampled, y_train_resampled)
dt_resampled_pred = dt_resampled.predict(X_test)
print(collections.Counter(dt_resampled_pred))
print(balanced_accuracy_score(y_test, dt_resampled_pred))


Counter({0: 114, 1: 17})
0.9913793103448276


### Downsampling the major class

In [16]:
X_train_np = X_train.toarray()
class_0_indices = [i for i, x in enumerate(y_train==0) if x]
class_1_indices = [i for i, x in enumerate(y_train == 1) if x]
size_class_1 = sum(y_train == 1)
X_train_class_1 = X_train_np[class_1_indices, :]
y_train_class_1 = [1] * size_class_1
X_train_class_0 = X_train_np[class_0_indices, :]
X_train_class_0_downsampled = resample(
    X_train_class_0, replace=False, n_samples=size_class_1
)
y_train_class_0_downsampled = [0] * size_class_1

In [17]:
X_train_downsampled = np.concatenate([X_train_class_1, X_train_class_0_downsampled])
y_train_downsampled = y_train_class_1 + y_train_class_0_downsampled

In [19]:
X_train_downsampled = sparse.csr_matrix(X_train_downsampled)
dt_downsampled = tree.DecisionTreeClassifier()
dt_downsampled.fit(X_train_downsampled, y_train_downsampled)
dt_downsampled_pred = dt_downsampled.predict(X_test)
print(collections.Counter(dt_downsampled_pred))
print(balanced_accuracy_score(y_test, dt_downsampled_pred))

Counter({0: 114, 1: 17})
0.9913793103448276


### classifier including inner balancing samplers

In [21]:
from imblearn.ensemble import BalancedBaggingClassifier

balanced_clf = BalancedBaggingClassifier(
    base_estimator = tree.DecisionTreeClassifier(),
    sampling_strategy = "auto",
    replacement = True
)

balanced_clf.fit (X_train, y_train)
balanced_clf_pred = balanced_clf.predict(X_test)
print(collections.Counter(balanced_clf_pred))
print(balanced_accuracy_score(y_test, balanced_clf_pred))

Counter({0: 113, 1: 18})
0.9870689655172413


We start by loading in a predefined dataset (step 1) using the scipy.sparse.load_npz
loading function to load previously saved sparse matrices. Our next step is to train a basic
Decision Tree model on our data (step 2). To measure performance, we utilize the balanced
accuracy score, a measure that is often used in classification problems with imbalanced
datasets. By definition, balanced accuracy is the average of recall obtained on each class.
The best value is 1, whereas the worst value is 0.
In the following steps, we employ different techniques to tackle the class imbalance. Our
first approach is to utilize class weights to adjust our Decision Tree to an imbalanced
dataset (step 3). The balanced mode uses the values of y to automatically adjust weights
inversely proportional to the class frequencies in the input data as n_samples / (n_classes *
np.bincount(y)). In steps 4 to 7, we utilize upsampling to tackle class imbalance. This is the
process of randomly duplicating observations from the minority class in order to reinforce
the minority class's signal.
There are several methods for doing so, but the most common way is to simply resample
with replacements as we have done. The two main concerns with upsampling are that it
increases the size of the dataset and that it can lead to overfitting due to training on the
same sample numerous times. In steps 8 to 10, we down-sample our major class. This
simply means that we don't use all of the samples we have, but just enough so that we
balance our classes.
The main issue with this technique is that we are forced to use a smaller training set. Our
final approach, and the most sophisticated one, is to utilize a classifier that includes inner
balancing samplers, namely the BalancedBaggingClassifier from imbalanced-learn
(step 11). Overall, we see that every single one of our methods for tackling class imbalance
increased the balanced accuracy score.